In [1]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import col

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

#!hdfs dfs -ls
!hdfs dfs -copyFromLocal Asistencia.csv /tmp/
!hdfs dfs -copyFromLocal Evaluacion.csv /tmp/

copyFromLocal: `/tmp/Asistencia.csv': File exists
copyFromLocal: `/tmp/Evaluacion.csv': File exists


In [3]:
!hdfs dfs -ls /tmp/*csv

-rw-r--r--   1 root supergroup      93121 2021-05-26 15:40 /tmp/Asistencia.csv
-rw-r--r--   1 root supergroup     111944 2021-05-26 15:40 /tmp/Evaluacion.csv


In [4]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...
asis = SesionSpark.read.load('/tmp/Asistencia.csv',format="csv",sep=',',inferSchema='true',header='true')
eva = SesionSpark.read.load('/tmp/Evaluacion.csv',format="csv",sep=',',inferSchema='true',header='true')

In [5]:
#Validamos datos cambiando el nombre de las columnas
asis1 = asis.withColumnRenamed("Marca temporal", "tiempo")
asis2 = asis1.withColumnRenamed("Nombre de usuario", "usuario")
asis3 = asis2.withColumnRenamed("Equipo al que perteneces:", "equipoPertenece")
asis4 = asis3.withColumnRenamed("Equipo que va a exponer:", "equipoExponer")
asis4.printSchema()
asis = asis4

root
 |-- tiempo: string (nullable = true)
 |-- usuario: string (nullable = true)
 |-- equipoPertenece: string (nullable = true)
 |-- equipoExponer: string (nullable = true)



In [6]:
eva2 = eva.withColumnRenamed("Marca temporal","tiempo") \
    .withColumnRenamed("Nombre de usuario","usuario") \
    .withColumnRenamed("Equipo que vas a evaluar:","equipoEvaluar") \
    .withColumnRenamed("Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?","ventajas") \
    .withColumnRenamed("Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?","problema") \
    .withColumnRenamed("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?","introduccion") \
    .withColumnRenamed("Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?","solucion") \
    .withColumnRenamed("Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.","traccion") \
    .withColumnRenamed("Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ","modeloNegocio") \
    .withColumnRenamed("Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?","inversion") \
    .withColumnRenamed("Exposición: ¿Qué tan coordinados estaban los expositores?","exposicion1") \
    .withColumnRenamed("Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?","exposicion2") \
    .withColumnRenamed("Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.","exposicion3") \
    .withColumnRenamed("Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)","supon") \
    .withColumnRenamed("Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación.","observacion") \
    .withColumnRenamed("Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.","contacto") \
    .withColumnRenamed("Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?","competencia") \
    .withColumnRenamed("Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.","mercado") \
    .withColumnRenamed("Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.","producto") \
    .withColumnRenamed("Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?", "equipo")
eva = eva2
eva2.printSchema()

root
 |-- tiempo: string (nullable = true)
 |-- usuario: string (nullable = true)
 |-- equipoEvaluar: string (nullable = true)
 |-- introduccion: integer (nullable = true)
 |-- equipo: integer (nullable = true)
 |-- problema: integer (nullable = true)
 |-- ventajas: integer (nullable = true)
 |-- solucion: integer (nullable = true)
 |-- producto: integer (nullable = true)
 |-- traccion: integer (nullable = true)
 |-- mercado: integer (nullable = true)
 |-- competencia: integer (nullable = true)
 |-- modeloNegocio: integer (nullable = true)
 |-- inversion: integer (nullable = true)
 |-- contacto: integer (nullable = true)
 |-- exposicion1: integer (nullable = true)
 |-- exposicion2: integer (nullable = true)
 |-- exposicion3: integer (nullable = true)
 |-- supon: string (nullable = true)
 |-- observacion: string (nullable = true)



In [7]:
#Datos adicionales
totalAsis = asis.count()
totalEva = eva.count()
print('Total de asistencias: ', totalAsis, ' y total de evaluaciones: ', totalEva, '.')

Total de asistencias:  662  y total de evaluaciones:  617 .


In [8]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

#um = asis.sort(['usuario']).select(['usuario']).drop_duplicates().count()
#Respuesta
asis.sort(['usuario']).select(['usuario']).drop_duplicates().show()

+--------------------+
|             usuario|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
|cquinchiar@unal.e...|
| dadazam@unal.edu.co|
|daestradam@unal.e...|
|davgarciava@unal....|
|dballesteroso@una...|
|  dbrito@unal.edu.co|
|dcadavid@unal.edu.co|
|dcardonaal@unal.e...|
|dchavarriar@unal....|
|dgarciabl@unal.ed...|
+--------------------+
only showing top 20 rows



In [9]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

################ FALTA ESTE PUNTO ############################
from pyspark.sql import functions as F
asis.groupBy('usuario').agg(F.count('usuario')).show()

+--------------------+--------------+
|             usuario|count(usuario)|
+--------------------+--------------+
|samolinap@unal.ed...|            10|
| jruedat@unal.edu.co|             8|
|nvalenciat@unal.e...|             8|
|serendona@unal.ed...|             8|
|jgutierrezlo@unal...|             8|
|dgerenal@unal.edu.co|             8|
|cjfunezg@unal.edu.co|             8|
|emflorezb@unal.ed...|             8|
|dchavarriar@unal....|             8|
|fguerrerot@unal.e...|             8|
|jdroldano@unal.ed...|             1|
|juriveras@unal.ed...|             8|
|jovillarrealm@una...|            10|
|dguardia@unal.edu.co|             8|
| lrdiaza@unal.edu.co|             7|
|fmiranda@unal.edu.co|             8|
|dcadavid@unal.edu.co|             8|
| bocampo@unal.edu.co|             8|
|rmbuilesm@unal.ed...|             9|
|jmunozhe@unal.edu.co|             8|
+--------------------+--------------+
only showing top 20 rows



In [10]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...

#Creamos una vista para trabajar con SQL
asis.createOrReplaceTempView('vistaAsis')

In [11]:
#Respuesta
SesionSpark.sql("""
 select usuario, total from (select usuario, count(usuario) as total from vistaAsis group by usuario) where total <= 2
""").show()

+--------------------+-----+
|             usuario|total|
+--------------------+-----+
|jdroldano@unal.ed...|    1|
+--------------------+-----+



In [12]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...

#Creamos otra vista para trabajar con SQL
eva.createOrReplaceTempView('vistaEva')

In [13]:
#Respuesta
SesionSpark.sql("""
 select E.usuario from vistaEva E left join vistaAsis A on E.usuario = A.usuario where A.usuario is null
""").show()

+--------------------+
|             usuario|
+--------------------+
| que genera un có...|
|                null|
|                null|
| o las imágenes c...|
| hasta más famoso...|
| poder ver gráfic...|
|                null|
| ya que no se le ...|
|                null|
|                null|
| parece publicida...|
|                null|
|                null|
| como todo buen e...|
|                null|
|                null|
|    UberEats incluso|
| pueden incluir a...|
| por el hecho de ...|
| es un gasto que ...|
+--------------------+
only showing top 20 rows



In [14]:
SesionSpark.sql("""
 select E.usuario from vistaEva E left join vistaAsis A on E.usuario = A.usuario where A.usuario is null
""").count()

43

In [15]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 select equipoPertenece, count(usuario) from vistaAsis group by equipoPertenece
""").show()

+--------------------+--------------+
|     equipoPertenece|count(usuario)|
+--------------------+--------------+
|Pertenezco a: Ges...|            84|
|Pertenezco a: Lo ...|            87|
|Pertenezco a: Adm...|            95|
|Pertenezco a: Ser...|            65|
|Pertenezco a: Te ...|            89|
|Pertenezco a: Ges...|            86|
|Pertenezco a: Ges...|            73|
|Pertenezco a: Mi ...|            83|
+--------------------+--------------+



In [16]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 select equipoPertenece, count(usuario) from vistaAsis where equipoPertenece <> equipoExponer group by equipoPertenece 
""").show()

+--------------------+--------------+
|     equipoPertenece|count(usuario)|
+--------------------+--------------+
|Pertenezco a: Ges...|            84|
|Pertenezco a: Lo ...|            87|
|Pertenezco a: Adm...|            95|
|Pertenezco a: Ser...|            65|
|Pertenezco a: Te ...|            89|
|Pertenezco a: Ges...|            86|
|Pertenezco a: Ges...|            73|
|Pertenezco a: Mi ...|            83|
+--------------------+--------------+



In [17]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 select * from (select A.usuario, SUBSTRING(A.equipoPertenece, 14) AS perteneceA, E.equipoEvaluar from vistaAsis A inner join vistaEva E on A.usuario = E.usuario) where perteneceA = equipoEvaluar
""").show()

+-------+----------+-------------+
|usuario|perteneceA|equipoEvaluar|
+-------+----------+-------------+
+-------+----------+-------------+



In [18]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 select
 avg(introduccion) as promIntroduccion,
 avg(equipo) as promEquipo,
 avg(problema) as promProblema,
 avg(ventajas) as promVentajas,
 avg(solucion) as promSolucion,
 avg(producto) as promProducto,
 avg(traccion) as promTraccion,
 avg(mercado) as promMercado,
 avg(competencia) as promCompetencia,
 avg(modeloNegocio) as PromModeloNegiocio,
 avg(inversion) as promInversion,
 avg(contacto) as promContacto,
 avg(exposicion1) as promExposicion1,
 avg(exposicion2) as promExposicion2,
 avg(exposicion3) as promExposicion3 
 from vistaAsis  
""").show()

#introduccion, equipo, problema, ventajas, solucion, producto, traccion, mercado, competencia, modeloNegocio, inversion, contacto, exposicion1, exposicion2, exposicion3


AnalysisException: "cannot resolve '`introduccion`' given input columns: [vistaasis.tiempo, vistaasis.usuario, vistaasis.equipoPertenece, vistaasis.equipoExponer]; line 3 pos 5;\n'Project ['avg('introduccion) AS promIntroduccion#646, 'avg('equipo) AS promEquipo#647, 'avg('problema) AS promProblema#648, 'avg('ventajas) AS promVentajas#649, 'avg('solucion) AS promSolucion#650, 'avg('producto) AS promProducto#651, 'avg('traccion) AS promTraccion#652, 'avg('mercado) AS promMercado#653, 'avg('competencia) AS promCompetencia#654, 'avg('modeloNegocio) AS PromModeloNegiocio#655, 'avg('inversion) AS promInversion#656, 'avg('contacto) AS promContacto#657, 'avg('exposicion1) AS promExposicion1#658, 'avg('exposicion2) AS promExposicion2#659, 'avg('exposicion3) AS promExposicion3#660]\n+- SubqueryAlias `vistaasis`\n   +- Project [tiempo#68, usuario#73, equipoPertenece#78, Equipo que va a exponer:#13 AS equipoExponer#83]\n      +- Project [tiempo#68, usuario#73, Equipo al que perteneces:#12 AS equip

In [19]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...


In [20]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...



In [21]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

